In [1]:
!pip install deap
!pip install graphviz

In [2]:
import random
import operator
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, gp
import multiprocessing

# Load and preprocess the data
df = pd.read_csv('lungcancer_binario.csv')

def convert_to_binary(df):
    df_binary = df.copy()

    for column in df_binary.columns:
        # Verifica si la columna es binaria (0/1)
        if df_binary[column].dtype in ['int64', 'float64']:
            if df_binary[column].nunique() == 2:
                df_binary[column].astype(int)  # No modificar columnas binarias
            elif df_binary[column].nunique() > 3:
                # Aplica binning y luego one-hot
                df_binned = pd.cut(df_binary[column], bins=3, labels=False)
                df_one_hot = pd.get_dummies(df_binned, prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1).astype(int)
                df_binary.drop(column, axis=1, inplace=True)
            else:
                # Aplica one-hot directamente
                df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1).astype(int)
                df_binary.drop(column, axis=1, inplace=True)
        elif df_binary[column].dtype == 'object':
            # Si la columna es categórica, aplica one-hot
            df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
            df_binary = pd.concat([df_binary, df_one_hot], axis=1)
            df_binary.drop(column, axis=1, inplace=True)

    return df_binary
# Convert categorical variables to binary (one-hot encoding)
df_bin = convert_to_binary(df)
df_bin.columns

FileNotFoundError: [Errno 2] No such file or directory: '/content/lungcancer_binario.csv'

In [ ]:
# Separate features and target
X = df_bin.drop(['CancerPulmon'], axis=1)
Y = df_bin['CancerPulmon']

# Convert DataFrames to NumPy arrays for faster computations
X = X.values.astype(int)
Y = Y.values.astype(int)

# Split into train and test sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

# Define logical functions
def logical_and(a, b):
    return int(a and b)

def logical_or(a, b):
    return int(a or b)

def logical_not(a):
    return int(not a)

# Number of variables (features)
# Con esto creamos x1, x2, x3... almacenados en una lista
num_vars = Xtrain.shape[1]
var_names = ['x' + str(i+1) for i in range(num_vars)]

# Create PrimitiveSet
# Se crea el conjunto de primitivas "MAIN" (el conjunto de funciones y operadores que
# sirven para construir inidividuos) con el tamaño del número de características
# que tenemos en el problema
pset = gp.PrimitiveSet("MAIN", num_vars)
# Se renombran los argumentos (por defecto llamados ARG0,ARG1...) para que se llamen (x1,x2...)
pset.renameArguments(**{'ARG' + str(i): var_names[i] for i in range(num_vars)})

# Add primitives to the set
# Añade las funciones al conjunto de primitivas
pset.addPrimitive(logical_and, 2)
pset.addPrimitive(logical_or, 2)
pset.addPrimitive(logical_not, 1)

# Terminal set can include constants if needed (e.g., True/False)
# pset.addTerminal(1)  # Optional: Add constant 'True'
# pset.addTerminal(0)  # Optional: Add constant 'False'

# Define fitness and individual classes
# Se crea una clase FitnessMax que maximiza el fitness de los individuos
# Se define una clase "Individual" que representa a los individuos de la población
# (Se obliga a que sean árboles formados por las primitivas de antes)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

# Create toolbox
# Se crea una toolbox que contiene las funciones (herramientas) que se van a usar
# para generar individuos poblaciones...
toolbox = base.Toolbox()
# Función para generar expresiones (gp.genHalfAndHalf sirve para crear árboles
# utilizando crecimiento completo=hasta la máxima longitud y crecimiento
#parcial=árboles de tamaño libre)
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
# Función para crear individuos (usando la anterior función)
toolbox.register("individual", tools.initIterate,
                 creator.Individual, toolbox.expr)
# Función para crear población, llamando repetidamente a la función anterior
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register compile function
# Función para convertir un individuo (su árbol) en una función de Python ejecutable
# esto sirve para poder evaluar a los individuos
toolbox.register("compile", gp.compile, pset=pset)

# Evaluation function
# Para medir el f1-score
def eval_individual(individual):
    func = toolbox.compile(expr=individual)
    predictions = []
    for i in range(len(Xtrain)):
        args = tuple(Xtrain[i])
        pred = func(*args)
        predictions.append(pred)
    y_pred = np.array(predictions)
    y_true = Ytrain
    # Compute F1 score
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / \
        (precision + recall) if (precision + recall) > 0 else 0
    return f1,

# Register evaluation function
# Para añadir la función anterior al conjunto de herramientas
toolbox.register("evaluate", eval_individual)

# Genetic operators
# Funciones de selección por torneos, cruce (en un punto), generación de árboles
#completos y mutación uniforme del árbol generado
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Limit the height of the tree to prevent bloat
# Estas funciones limitan, tanto el cruce como la mutación para que los árboles
# resultantes no superen la altura definida
# Se podrían eliminar para no limitar el crecimiento de los árboles
MAX_HEIGHT = 5
MAX_SIZE = 50  # Ajusta este valor según tus necesidades
toolbox.decorate("mate", gp.staticLimit(
    key=operator.attrgetter("height"), max_value=MAX_HEIGHT))
toolbox.decorate("mate", gp.staticLimit(key=len, max_value=MAX_SIZE))
toolbox.decorate("mutate", gp.staticLimit(
    key=operator.attrgetter("height"), max_value=MAX_HEIGHT))
toolbox.decorate("mutate", gp.staticLimit(key=len, max_value=MAX_SIZE))
def main():
    random.seed(42)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    # Statistics
    # Métricas sobre el tamaño del inviduo y de su fitness
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    # Use multiprocessing for parallel evaluations
    # Para optimizar el proceso evolutivo
    pool = multiprocessing.Pool()
    toolbox.register("map", pool.map)

    # Evaluate the entire population
    # Evalúa en paralelo los individuos
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the initial population
    hof.update(pop)

    ngen = 150
    for gen in range(ngen):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop) - 1)
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:  # Curce con prob = 50%
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2: # Mutación con prob = 20%
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # Evaluate the individuals with invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        # Replace the population with the offspring, keeping the best individual
        pop[:] = offspring + [hof[0]]
        # Update the hall of fame
        hof.update(pop)
        # Record statistics
        record = mstats.compile(pop)
        print(f"Gen {gen}: Max F1 Score = {record['fitness']['max']:.4f}")
    pool.close()
    pool.join()
    return pop, hof

def evaluate_on_test_set(individual):
    func = toolbox.compile(expr=individual)
    predictions = []
    for i in range(len(Xtest)):
        args = tuple(Xtest[i])
        pred = func(*args)
        predictions.append(pred)
    y_pred = np.array(predictions)
    y_true = Ytest
    # Compute F1 score
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / \
        (precision + recall) if (precision + recall) > 0 else 0
    return f1

if __name__ == "__main__":
    pop, hof = main()
    best_individual = hof[0]
    print("\nBest individual:")
    print(best_individual)
    print("\nBest training F1 Score:", best_individual.fitness.values[0])
    test_f1 = evaluate_on_test_set(best_individual)
    print("Test F1 Score:", test_f1)

In [ ]:
best_individual = hof[0]
print("\nBest individual:")
print(best_individual)
print("\nBest training F1 Score:", best_individual.fitness.values[0])
test_f1 = evaluate_on_test_set(best_individual)
print("Test F1 Score:", test_f1)

In [ ]:
import matplotlib.pyplot as plt
from deap import gp
import graphviz

# Plot the tree
nodes, edges, labels = gp.graph(best_individual)

# Create a new graph object using graphviz
g = graphviz.Digraph(format='png')

# Add nodes to the graph
for node in nodes:
    g.node(str(node), labels[node])

# Add edges to the graph
for edge in edges:
    g.edge(str(edge[0]), str(edge[1]))

# Render and save the graph
g.render('best_individual_tree', view=True)
